# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
# DEPLOYMENT_URL="http://localhost:2024/"
# DEPLOYMENT_URL="http://localhost:8000/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data
 
 
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)
 
        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    if msg_type == "ai":
                        return True
    return False

In [3]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': 'f0df0608-fa69-4cd4-babd-c04606f22ac2', 'created_at': '2025-08-14T08:50:09.249671+00:00', 'updated_at': '2025-08-14T08:50:09.249671+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None, 'interrupts': {}, 'error': None}
f0df0608-fa69-4cd4-babd-c04606f22ac2


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [4]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [5]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [6]:
input = { "messages": "제주도 여행"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    list_chunk.append(chunk)
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])
    # elif filter_silde_data(chunk):
    #     print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
    #     print(chunk)


##########  AI 메시지  ##########
제주도 여행을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 제주도의 관광 명소 소개 - 제주도에서 꼭 방문해야 할 유명 관광지를 소개합니다. 제주의 자연과 문화가 어우러진 대표적인 장소들을 중심으로 여행 계획을 세워보세요. 
3. 제주도의 맛있는 먹거리 - 제주도에서 맛볼 수 있는 다양한 먹거리를 추천합니다. 흑돼지, 성게미역국 등의 향토 요리부터 한식과 해산물 요리까지, 제주의 맛을 즐겨보세요. 
4. 제주도의 교통편 정보 - 제주도 여행 시 편리한 교통편을 선택하는 방법에 대해 안내합니다. 대중교통, 렌터카, 그리고 자전거와 같은 이동 수단을 고려하여 효율적인 여행을 계획하세요. 
5. 제주도 여행을 위한 팁 - 제주도를 더욱 알차게 즐길 수 있는 유용한 여행 팁을 제공하여 여행 계획의 효율성을 높입니다. 계절에 맞는 추천 방문 시기, 예산 절약 방법 등을 소개합니다. 
6. 제주도의 추천 숙소 - 제주도에서 방문객을 위한 다양한 숙소 추천 리스트를 제공합니다. 리조트, 호텔, 펜션, 풀빌라 등 다양한 선택지를 통해 여행 기간 동안 편안한 휴식을 취할 수 있습니다. 



RemoteProtocolError: peer closed connection without sending complete message body (incomplete chunked read)

In [ ]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [ ]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

In [ ]:
len(state["values"]["slides"])

In [ ]:
state["values"]["slides"]

In [ ]:
for s in state["values"]["slides"]:
    with open(f"{s['idx']}.html", "w", encoding="utf-8") as f:
        f.write(s['html'])